In [4]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random

def plot_rect(big, smalls):
  big_rectangle    = [big[0], big[1], big[2]-big[0], big[3]-big[1]]
  small_rectangles = [[i[0], i[1], i[2]-i[0], i[3]-i[1]] for i in smalls]
  
  fig, ax = plt.subplots()

  big_rect_patch = patches.Rectangle(
      (big_rectangle[0], big_rectangle[1]),
      big_rectangle[2],
      big_rectangle[3],
      linewidth=1,
      edgecolor='black',
      facecolor='none'
  )
  ax.add_patch(big_rect_patch)

  for rect_coords in small_rectangles:
      random_color = random.choice(['blue', 'green', 'yellow', 'red'])  # Choose random colors
      small_rect_patch = patches.Rectangle(
          (rect_coords[0], rect_coords[1]),
          rect_coords[2],
          rect_coords[3],
          linewidth=1,
          edgecolor='black',
          facecolor=random_color
      )
      ax.add_patch(small_rect_patch)

  colors = ['blue', 'green', 'yellow', 'red']
  color_labels = [random.choice(colors) for _ in range(len(small_rectangles))]

  color_patches = [patches.Patch(color=color) for color in color_labels]
  ax.set_aspect('equal')
  ax.set_xlim(0, big_rectangle[2])
  ax.set_ylim(0, big_rectangle[3]) 
  plt.show()

In [82]:
class Guillotine:
  def __init__(self, H, W, ratios):
    self.ratios = sorted(ratios, reverse=True)
    self.H = H
    self.W = W
    self.coords=[]
  
  def plot(self):
    big = [0, 0, self.W, self.H]
    smalls = self.coords
    plot_rect(big, smalls)

  def diff(self, pair, r):
    return abs(max(pair)/min(pair)-r)
  
  def diff_rect(self, rect, r):
    xl, yl, xr, yr = rect
    W, H = xr-xl+1, yr-yl+1
    return abs(max(W,H)/min(W,H)-r)
  
  def del_emptys(self, rects):
    res = []
    for r in rects:
      diffx, diffy = r[2]-r[0], r[3]-r[1]
      if diffx<-1 or diffy<-1: 
        return []
      elif diffx>0 and diffy>0:
        res.append(r)
    return res
  
  def split_iteration(self, coords, r, areac_onstr): # areac_onstr = (globH*globW - sumr)/r**2
    xl, yl, xr, yr = map(int, coords)
    W, H = xr-xl+1, yr-yl+1
    # split using the biggest ratio r1
    # take n1 with the maximum value s.t. HW <= n1*r1^2+r2+...
    # also n1r1<=max(H,W), n1<=max(H,W)
    n1 = min(areac_onstr, max(H, W)/r, H, W)

    # adjust n1, n1*r1 to integers keeping ratio:
    n1 = int(max(n1, 1))
    k=1
    print(n1, int(n1*r))
    while (self.diff((n1, bigger:=int(n1*r)), r)>=0.1) and (n1>0) and (bigger<=max(H, W)):
      print('A')
      n1 -= k
      k = -k - 1 if k>0 else -k + 1
    bigger = int(n1*r)
    assert(self.diff((n1, int(n1*r)), r)<0.1 and n1>=0 and n1<=min(areac_onstr, H, W) and bigger<=max(H, W))
    

    # now orient it in the way that it could fill bigger side
    # place it in the left lower corner
    is_horizontal = H-bigger<0 or (W-bigger <= H-bigger)

    if is_horizontal:
      rects1 = [(xl+bigger, yl, xr, yr), (xl, yl+n1, xl+bigger-1, yr)] # vertical
      rects2 = [(xl+bigger, yl, xr, yl+n1-1), (xl, yl+n1, xr, yr)]     # hor line
      placed_coords = (xl,yl, xl+bigger, yl+n1)
    else:
      rects1 = [(xl+n1, yl, xr, yr), (xl, yl+bigger, xl+n1-1, yr)]
      rects2 = [(xl+n1, yl, xr, yl+bigger-1), (xl, yl+bigger, xr, yr)]
      placed_coords = (xl,yl, xl+n1, yl+bigger)
    
    # if not self.del_emptys(rects1)+self.del_emptys(rects2):
    print(n1, bigger, rects1, rects2, placed_coords, W, H)

    return self.del_emptys(rects1), self.del_emptys(rects2), placed_coords, n1*bigger

  def iterate(self):
    Fs = [(0, 0, self.W-1, self.H-1)]
    Area = self.W*self.H
    while self.ratios:
      r = self.ratios.pop(0)
      area_constr = (Area - sum(self.ratios))/r**2

      # split and update area and coords
      Fs0 = Fs.pop(0)
      Fs1, Fs2, placed_coords, area = self.split_iteration(Fs0, r, area_constr)
      Area -= area
      self.coords.append(placed_coords)

      if not Fs1 or not Fs2:
        Fs += Fs1+Fs2
      elif self.ratios: # compute which split is better by best next ratio:
        nextr = self.ratios[0]
        if self.diff_rect(Fs1[0],nextr)<=self.diff_rect(Fs2[0],nextr):
          Fs += Fs1
        else:
          Fs += Fs2
      # sort rects by fit ratio:
      Fs.sort(key=lambda x: self.diff_rect(x,r))

H, W = train[0][:2]
ratios = train[0][2:]
g = Guillotine(H,W,ratios)
g.iterate()

12 87
12 87 [(12, 0, 88, 86), (0, 87, 11, 86)] [(12, 0, 88, 86), (0, 87, 88, 86)] (0, 0, 12, 87) 89 87
16 83
16 83 [(95, 0, 88, 86), (12, 16, 94, 86)] [(95, 0, 88, 15), (12, 16, 88, 86)] (12, 0, 95, 16) 77 87


IndexError: pop from empty list

In [83]:
with open('task1.csv', 'r') as f:
  train = f.readlines()[1:]
  train = [list(map(float, i[:-1].split(','))) for i in train]

H, W = train[0][:2]
ratios = train[0][2:]
g = Guillotine(H,W,ratios)
g.iterate()
g.plot()

12 87
12 87 [(12, 0, 88, 86), (0, 87, 11, 86)] [(12, 0, 88, 86), (0, 87, 88, 86)] (0, 0, 12, 87) 89 87
16 83
16 83 [(95, 0, 88, 86), (12, 16, 94, 86)] [(95, 0, 88, 15), (12, 16, 88, 86)] (12, 0, 95, 16) 77 87


IndexError: pop from empty list

In [79]:
92/14

6.571428571428571